<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/DNN/DNN_TensorFlow_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Neural Networks com TensorFlow 
## Regressão

![TensorFlow](https://github.com/vladimiralencar/DeepLearning-LANA/blob/master/DNN/images/tensorflow.png?raw=true "TensorFlow")

TensorFlow é uma biblioteca de software de código aberto para aprendizagem de máquinas em vários tipos de tarefas de compreensão de percepção e linguagem. Atualmente, ele é usado tanto para pesquisa quanto para produção por diferentes equipes em muitos produtos comerciais do Google, como reconhecimento de fala, Gmail, Google Photos e pesquisas, muitas das quais anteriormente utilizaram seu antecessor DistBelief. O TensorFlow foi originalmente desenvolvido pela equipe do Google Brain para fins de pesquisa e produção do Google e posteriormente lançado sob a licença de código aberto Apache 2.0 em 9 de novembro de 2015.

* [TensorFlow Homepage](https://www.tensorflow.org/)
* [TensorFlow GitHib](https://github.com/tensorflow/tensorflow)
* [TensorFlow Google Groups Support](https://groups.google.com/forum/#!forum/tensorflow)
* [TensorFlow Google Groups Developer Discussion](https://groups.google.com/a/tensorflow.org/forum/#!forum/discuss)
* [TensorFlow FAQ](https://www.tensorflow.org/resources/faq)


In [1]:
import tensorflow as tf
print("versão do TensorFlow : {}".format(tf.__version__))

versão do TensorFlow : 1.12.0


# Usando TensorFlow

TensorFlow é uma API de matemática de baixo nível, semelhante ao [Numpy] ( http://www.numpy.org/ ). No entanto, ao contrário de Numpy, TensorFlow é construído para aprendizagem profunda. O TensorFlow funciona permitindo que você defina grafos de computação com o Python. O TensorFlow compila esses grafos de computação em um código C ++ / [CUDA] (https://developer.nvidia.com/cuda-zone) altamente eficiente.

Neste exercício, seu trabalho é criar as camadas da rede neural, necessárias para construir o modelo de regressão, usando TensorFlow e Keras. Usaremos o dataset auto-mpg.csv e nosso objetivo é prever o MPG (Milhas por Galão - algo similar a Km/Litro) dos automóveis. Por se tratar de um valor numérico, regressão é a melhor solução nesse caso!

### Funções de Transformação (função de apoio para construção do modelo)

In [0]:
# Função para preencher os valores NA
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)
    
# Converte um dataframe do Pandas para inputs x,y que o TensorFlow precisa
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    
    # Descubrindo o tipo da coluna de destino.  
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    
    # Codificação. TensorFlow gosta de 32 bits.
    if target_type in (np.int64, np.int32):
        # Classificação
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regressão
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# TensorFlow Regression: MPG

Este exemplo mostra como codificar o conjunto de dados MPG para regressão. Isso é um pouco mais complexo que classificação, porque:

* A entrada tem valores numéricos e categóricos
* A entrada tem valores em falta (missing)

Para codificar valores categóricos que fazem parte do vetor de características, use as funções de ajuda acima. 

### Carregando os dados

In [6]:
from google.colab import files
files.upload()
!mkdir data
!mv *.csv data

Saving auto-mpg.csv to auto-mpg (1).csv
mkdir: cannot create directory ‘data’: File exists


In [8]:
!ls data

'auto-mpg (1).csv'   auto-mpg.csv


In [9]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

# Define o PATH onde estão os dados
path = "./data"

# Carrega o dataset
filename_read = os.path.join(path,"auto-mpg.csv")

# Preenche com o valor NA quando não houver dados na coluna
df = pd.read_csv(filename_read, na_values = ['NA','?'])

# Separa a variável name
cars = df['name']

# A variável name não é relevante no dataset. Removemos.
df.drop('name', 1, inplace = True)

# Preenchemos os valores NA com a mediana da coluna horsepower
missing_median(df, 'horsepower')

# Convertemos o input para x e y
x,y = to_xy(df,"mpg")

Using TensorFlow backend.


In [10]:
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
2,18.0,8,318.0,150.0,3436,11.0,70,1
3,16.0,8,304.0,150.0,3433,12.0,70,1
4,17.0,8,302.0,140.0,3449,10.5,70,1


In [12]:
np.set_printoptions(precision=5, suppress=True)
x[:3]

array([[   8. ,  307. ,  130. , 3504. ,   12. ,   70. ,    1. ],
       [   8. ,  350. ,  165. , 3693. ,   11.5,   70. ,    1. ],
       [   8. ,  318. ,  150. , 3436. ,   11. ,   70. ,    1. ]],
      dtype=float32)

In [13]:
y[:3]

array([[18.],
       [15.],
       [18.]], dtype=float32)

In [14]:
print(len(x), len(y))
print(x.shape)

398 398
(398, 7)


In [16]:
# Construindo a rede neural
model = Sequential()
model.add( Dense(10, input_dim =  x.shape[1], activation= 'relu'))
model.add( Dense(1, kernel_initializer = 'normal')) 
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 10)                80        
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________


In [17]:
# treinando a rede neural
model.fit(x,y,verbose = 2, epochs = 100)

Epoch 1/100
 - 0s - loss: 420.1421
Epoch 2/100
 - 0s - loss: 227.7126
Epoch 3/100
 - 0s - loss: 168.3149
Epoch 4/100
 - 0s - loss: 155.3047
Epoch 5/100
 - 0s - loss: 151.6968
Epoch 6/100
 - 0s - loss: 150.0041
Epoch 7/100
 - 0s - loss: 146.2981
Epoch 8/100
 - 0s - loss: 143.0810
Epoch 9/100
 - 0s - loss: 143.1372
Epoch 10/100
 - 0s - loss: 138.0618
Epoch 11/100
 - 0s - loss: 137.4946
Epoch 12/100
 - 0s - loss: 135.4890
Epoch 13/100
 - 0s - loss: 132.4612
Epoch 14/100
 - 0s - loss: 129.8300
Epoch 15/100
 - 0s - loss: 129.9638
Epoch 16/100
 - 0s - loss: 126.8119
Epoch 17/100
 - 0s - loss: 125.4602
Epoch 18/100
 - 0s - loss: 122.3539
Epoch 19/100
 - 0s - loss: 117.1600
Epoch 20/100
 - 0s - loss: 117.2210
Epoch 21/100
 - 0s - loss: 113.3834
Epoch 22/100
 - 0s - loss: 112.9288
Epoch 23/100
 - 0s - loss: 108.8562
Epoch 24/100
 - 0s - loss: 115.4622
Epoch 25/100
 - 0s - loss: 106.7578
Epoch 26/100
 - 0s - loss: 105.5045
Epoch 27/100
 - 0s - loss: 103.1408
Epoch 28/100
 - 0s - loss: 100.0329
E

Em seguida, iremos realizar previsões reais. Essas previsões são atribuídas à variável ** pred **. Estas são todas as previsões MPG da rede neural. Observe que esta é uma matriz 2D. Você sempre pode ver as dimensões do que é retornado ao imprimir ** pred.shape **. As redes neurais podem retornar valores múltiplos, então o resultado é sempre uma matriz. Aqui, a rede neural retorna apenas 1 valor por previsão (existem 398 carros, portanto, 398 previsões). No entanto, é necessária uma matriz 2D porque a rede neural possui o potencial de retornar mais de um valor.

In [18]:
pred = model.predict(x)
print("Shape: {}".format(pred.shape))
print(pred)

Shape: (398, 1)
[[13.21737]
 [ 8.7247 ]
 [10.51556]
 [11.53299]
 [12.19471]
 [ 4.1287 ]
 [ 0.80871]
 [ 1.73474]
 [ 0.88577]
 [ 4.5357 ]
 [ 5.49162]
 [ 8.6232 ]
 [ 6.82038]
 [-5.18098]
 [22.89559]
 [19.77259]
 [19.29183]
 [19.40968]
 [23.18828]
 [26.01842]
 [25.28048]
 [23.53119]
 [23.67469]
 [19.64028]
 [19.16027]
 [ 8.95626]
 [12.37518]
 [10.67907]
 [15.4053 ]
 [23.49019]
 [21.05335]
 [22.34514]
 [22.98902]
 [16.2695 ]
 [20.44694]
 [18.84967]
 [19.65183]
 [19.73245]
 [11.46444]
 [ 8.78128]
 [12.38031]
 [14.20765]
 [11.62988]
 [10.54414]
 [11.94406]
 [15.54171]
 [23.79227]
 [18.53676]
 [18.71338]
 [21.9308 ]
 [21.80793]
 [26.42281]
 [24.38587]
 [26.0878 ]
 [24.79541]
 [24.93734]
 [25.10263]
 [23.1735 ]
 [24.32325]
 [28.50409]
 [22.80744]
 [22.75973]
 [12.06086]
 [ 8.82521]
 [14.78624]
 [12.46898]
 [13.11969]
 [ 5.48506]
 [14.16421]
 [13.57058]
 [ 7.93722]
 [25.27208]
 [14.3164 ]
 [16.91236]
 [17.7267 ]
 [14.62341]
 [23.88801]
 [25.44926]
 [27.08149]
 [26.01372]
 [23.42546]
 [24.69965]


Gostaríamos de ver quão boas são essas previsões. Sabemos qual é o MPG correto para cada carro, então podemos medir a proximidade da rede neural.

In [19]:
# Usamos a métrica de erro RMSE. O RMSE é comum para a regressão.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 5.300751686096191


Isso significa que, em média, as previsões estavam dentro de +/- 6,24 valores do valor correto. Isso não é muito bom, mas logo veremos como melhorá-lo.

Nós também podemos imprimir os primeiros 10 carros, com previsões e MPG real.

In [20]:
# Previsões
for i in range(10):
    print("{}. Nome do Carro: {}, MPG Real: {}, MPG Previsto: {}".format(i+1,cars[i],y[i],pred[i]))

1. Nome do Carro: chevrolet chevelle malibu, MPG Real: [18.], MPG Previsto: [13.21737]
2. Nome do Carro: buick skylark 320, MPG Real: [15.], MPG Previsto: [8.7247]
3. Nome do Carro: plymouth satellite, MPG Real: [18.], MPG Previsto: [10.51556]
4. Nome do Carro: amc rebel sst, MPG Real: [16.], MPG Previsto: [11.53299]
5. Nome do Carro: ford torino, MPG Real: [17.], MPG Previsto: [12.19471]
6. Nome do Carro: ford galaxie 500, MPG Real: [15.], MPG Previsto: [4.1287]
7. Nome do Carro: chevrolet impala, MPG Real: [14.], MPG Previsto: [0.80871]
8. Nome do Carro: plymouth fury iii, MPG Real: [14.], MPG Previsto: [1.73474]
9. Nome do Carro: pontiac catalina, MPG Real: [14.], MPG Previsto: [0.88577]
10. Nome do Carro: amc ambassador dpl, MPG Real: [15.], MPG Previsto: [4.5357]


# Carregando/Salvando o Modelo Treinado

Quando a rede neural é criada, devemos especificar um diretório no qual a rede neural é salva. Muitas vezes, especificamos esse diretório para que o treinamento possa começar de novo. Esta definição é realizada chamando o método ** get_model_dir ** com o segundo parâmetro igual a TRUE. O código abaixo treina uma rede neural do MPG, exatamente como antes, salva e depois carrega para ser reutilizado.

In [0]:
!rm -rf save

In [24]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
path = "./data"
save_path = "./save"

!mkdir save

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read, na_values = ['NA','?'])

cars = df['name']
df.drop('name',1,inplace=True)
missing_median(df, 'horsepower')
x,y = to_xy(df,"mpg")

model = Sequential()
model.add( Dense(10, input_dim =  x.shape[1], activation= 'relu'))
model.add( Dense(1, kernel_initializer = 'normal')) 
model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model.fit(x, y, verbose = 0, epochs = 100)

# Previsões
pred = model.predict(x)

# Usamos a métrica de erro RMSE. O RMSE é comum para a regressão.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("Score antes de salvar o modelo (RMSE): {}".format(score))

# Salvando a estrutura da rede em formato JSON (sem os pesos)
model_json = model.to_json()
with open(os.path.join(save_path,"network.json"), "w") as json_file:
    json_file.write(model_json)

# Salvando a estrutura da rede em formato YAML (sem os pesos)
model_yaml = model.to_yaml()
with open(os.path.join(save_path,"network.yaml"), "w") as yaml_file:
    yaml_file.write(model_yaml)

# Salvando a rede inteira no formato HDF5 (salvando todos os componentes da rede, incluindo os pesos)
model.save(os.path.join(save_path,"network.h5"))
print("modelo salvo...")

Score antes de salvar o modelo (RMSE): 6.023092746734619
modelo salvo...


Agora recarregamos a rede e realizamos outra previsão. O RMSE deve corresponder ao anterior, exatamente se a rede neural foi realmente salva e recarregada.

In [25]:
from keras.models import load_model
model2 = load_model(os.path.join(save_path,"network.h5"))
pred = model2.predict(x)

score = np.sqrt(metrics.mean_squared_error(pred, y))
print("Score depois de carregar o modelo salvo (RMSE): {}".format(score))

Score depois de carregar o modelo salvo (RMSE): 6.023092746734619
